In [82]:
!pip3 install transformers
!pip3 install torch torchvision

In [83]:
!nvidia-smi

"nvidia-smi" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [84]:
from distutils.version import LooseVersion as LV

import io
import os
import numpy as np

#import matplotlib
#matplotlib.use('Agg')
#import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
import matplotlib as plt
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from sklearn.metrics import *
import numpy as np
import pandas as pd
from sklearn.metrics import matthews_corrcoef

In [85]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    devicename = '['+torch.cuda.get_device_name(0)+']'
else:
    device = torch.device('cpu')
    devicename = ""
    
print('Using PyTorch version:', torch.__version__,
      'Device:', device, devicename)
assert(LV(torch.__version__) >= LV("1.0.0"))

Using PyTorch version: 1.13.1+cpu Device: cpu 


C:\Users\bianc\AppData\Local\Temp\ipykernel_6020\1883626309.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  assert(LV(torch.__version__) >= LV("1.0.0"))


In [86]:
df = pd.read_csv("BERTdataset.csv")

In [87]:
df.head()

,Toxicity,tweet,tweet_clean,tweet_tokenized,tweet_tok=2,tweet_lemmatized,preprocessed_text
0,0,@user when a father is dysfunctional and is so...,father dysfunctional selfish drags kids dysfun...,"['father', 'dysfunctional', 'selfish', 'drags'...","[('father', 'dysfunctional'), ('dysfunctional'...","['father', 'dysfunctional', 'selfish', 'drag',...",father dysfunctional selfish drag kid dysfunct...
1,0,@user @user thanks for #lyft credit i can't us...,thanks lyft credit can not use cause do not ...,"['thanks', 'lyft', 'credit', 'can', 'not', 'us...","[('thanks', 'lyft'), ('lyft', 'credit'), ('cre...","['thank', 'lyft', 'credit', 'can', 'not', 'use...",thank lyft credit can not use cause do not off...
2,0,bihday your majesty,bihday majesty,"['bihday', 'majesty']","[('bihday', 'majesty')]","['bihday', 'majesty']",bihday majesty
3,0,factsguide: society now #motivation,factsguide society motivation,"['factsguide', 'society', 'motivation']","[('factsguide', 'society'), ('society', 'motiv...","['factsguide', 'society', 'motivation']",factsguide society motivation
4,0,[2/2] huge fan fare and big talking before the...,huge fan fare big talking leave chaos pay disp...,"['huge', 'fan', 'fare', 'big', 'talking', 'lea...","[('huge', 'fan'), ('fan', 'fare'), ('fare', 'b...","['huge', 'fan', 'fare', 'big', 'talk', 'leave'...",huge fan fare big talk leave chaos pay dispute...


In [140]:
# Encoding target variable
encoder = LabelEncoder()
X = df['tweet_clean']
#y = df['Toxicity']
y = encoder.fit_transform(df["Toxicity"]) 
# Split in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1000, stratify=y)

pandas.core.series.Series

In [89]:
print(y)

[0 0 0 ... 1 1 0]


In [90]:
# Transform target in tensors
train_labels = torch.tensor(y_train)
test_labels = torch.tensor(y_test)

TEXT PROCESSING PER BERT
# (1) Tokenize the sentence.
###  (3) Prepend the `[CLS]` token to the start. Append the `[SEP]` token to the end.
###   (2) Map tokens to their IDs.
###   (4) Pad or truncate the sentence to `max_length`
###   (5) Create attention masks for [PAD] tokens.
Function tokenizer.encode_plus was used which encapsulates the whole procedure

In [91]:
# (1)
# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [92]:
# Example for a tweet 
print("Tokenized tweet: ", tokenizer.tokenize(X_train.values[0]))
print("Original tweet: ", X_train.values[0])

Tokenized tweet:  ['hell', 'wrong', 'humanity', 'would', 'anyone', 'need', 'anything', 'prove']
Original tweet:  hell wrong humanity would anyone need anything prove


In [93]:
# (2)
# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(X_train.values[0])))

Token IDs:  [3109, 3308, 8438, 2052, 3087, 2342, 2505, 6011]


In [94]:
# (3)
# Example complete:
#To be properly handled by BERT we need to add the [CLS] token at the beginning of text. Il token [SEP] è un altro token speciale richiesto da BERT alla fine della frase.
sentence = X_train.values[0]
tokens_ = tokenizer.tokenize(sentence)
tokens_ = ['[CLS]'] + tokens_ + ['[SEP]']
print("Tokenized tweet: ", tokens_)
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokens_))

Tokenized tweet:  ['[CLS]', 'hell', 'wrong', 'humanity', 'would', 'anyone', 'need', 'anything', 'prove', '[SEP]']
Token IDs:  [101, 3109, 3308, 8438, 2052, 3087, 2342, 2505, 6011, 102]


RICERCA MAX LENGTH TRAIN & TEST

In [152]:
#(4)
#Ora impostiamo le lunghezze massime della sequenza per le nostre frasi di addestramento e test come MAX_LEN_TRAIN e MAX_LEN_TEST. 
#La lunghezza massima supportata dal modello BERT utilizzato è 128 e 512.
#Ma noi ricerchiamo la massima adatta al task --> prendo le frasi e le divido per trovare di quante parole sono composte --> è la lunghezza della frase 
#se lo facciamo per ogni frase poi vediamo quale è la più lunga e prendiamo come valore massimo quello
"""train = [len(i.split()) for i in X_train]
test = [len(i.split()) for i in X_test]

for i in [train, test]:
  pd.Series(i).hist(bins=20)"""

'train = [len(i.split()) for i in X_train]\ntest = [len(i.split()) for i in X_test]\n\nfor i in [train, test]:\n  pd.Series(i).hist(bins=20)'

In [158]:
MAX_LENGTH = 128
#encode train set
for i in X_train:
    encoded_data_train = tokenizer.batch_encode_plus(i,
                                                    add_special_tokens = True,
                                                    return_attention_mask = True,
                                                    pad_to_max_length = True,
                                                    max_length = MAX_LENGTH,
                                                    return_tensors = 'pt')
                                                
#encode validation set
for j in X_test:
    encoded_data_test = tokenizer.batch_encode_plus(j,
                                                    add_special_tokens = True,
                                                    return_attention_mask = True,
                                                    pad_to_max_length = True,
                                                    max_length = MAX_LENGTH,
                                                    return_tensors = 'pt')

c:\Users\bianc\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
